In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent
import re
import json

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Test

In [157]:
page_link = 'https://www.ipaddress.com/proxy-list/'
headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'
      }
response = requests.get(page_link, headers = headers)
proxy_df = pd.read_html(response.content)[0]

ha_proxy = proxy_df[proxy_df['Type']=='highly-anonymous']['Proxy'].values
tr_proxy = proxy_df[proxy_df['Type']=='transparent']['Proxy'].values

In [164]:
# url = 'https://energybase.ru/pipeline/espo-1-taishet-skovorodino'
ua = UserAgent()
headers = {'User-Agent': ua.random}
print(headers)
proxy = {'http':'http://' + ha_proxy[5]}
print(proxy)
response = requests.get(link, headers = headers, proxies=proxy)

In [170]:
driver.quit()

# get all pipelins

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_autoinstaller # pip install chromedriver-autoinstaller

# chromedriver_autoinstaller.install() # To update your chromedriver automatically
# driver = webdriver.Chrome()
driver = webdriver.Chrome()

# Get free proxies for rotating
def get_free_proxies(driver):
    driver.get('https://sslproxies.org')

    table = driver.find_element(By.TAG_NAME, 'table')
    thead = table.find_element(By.TAG_NAME, 'thead').find_elements(By.TAG_NAME, 'th')
    tbody = table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')

    headers = []
    for th in thead:
        headers.append(th.text.strip())

    proxies = []
    for tr in tbody:
        proxy_data = {}
        tds = tr.find_elements(By.TAG_NAME, 'td')
        for i in range(len(headers)):
            proxy_data[headers[i]] = tds[i].text.strip()
        proxies.append(proxy_data)
    
    return proxies


free_proxies = get_free_proxies(driver)

In [6]:
driver.quit()

In [9]:
from selenium.webdriver.chrome.options import Options

In [49]:
prx = free_proxies[1]
prx

{'IP Address': '201.229.250.19',
 'Port': '8080',
 'Code': 'DO',
 'Country': 'Dominican Republic',
 'Anonymity': 'elite proxy',
 'Google': 'no',
 'Https': 'yes',
 'Last Checked': '22 secs ago'}

In [47]:
PROXY = prx['IP Address'] + ':' + prx['Port']

In [48]:
ua = UserAgent()
userAgent = ua.random
options = Options()
# options.add_argument('--headless')
options.add_experimental_option ('excludeSwitches', ['enable-logging'])
options.add_argument("start-maximized")
options.add_argument('window-size=1920x1080')                                 
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')  
options.add_argument(f'user-agent={userAgent}') 
options.add_argument('--proxy-server={}'.format(PROXY))


driver = webdriver.Chrome(options=options)
driver.get('https://energybase.ru/pipeline/oil-pipeline')
ps = driver.page_source
soup = BeautifulSoup(ps,'html.parser')

In [50]:
driver.quit()

In [84]:
dict_links_pl = {}


for link in range(1,6):
    driver = webdriver.Chrome(chrome_options=chrome_options)
    link = f'https://energybase.ru/pipeline/oil-pipeline?page={link}'
    driver.get(link)
    ps = driver.page_source
    soup = BeautifulSoup(ps,'html.parser')

    tab_1 = soup.find_all('div',class_="object-card__header")
    for el in tab_1:
        my_val = el.contents[2].find('a', href=True).get('href')
        my_key = el.contents[2].find('a', href=True).text
        dict_links_pl[my_key] = my_val

    driver.quit()

with open('data/piple_href.json', 'w') as fp:
    json.dump(dict_links_pl, fp)

In [85]:
dict_links_pl = json.load(open('data/piple_href.json'))

# get data pipeline

In [51]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
# s = Service('/Users/vladimirkosarev/my_selenium/chromedriver')
# driver = webdriver.Chrome(service=s, options=chrome_options)
driver = webdriver.Chrome(chrome_options=chrome_options)

p1 = list(dict_links_pl.keys())[1]
pl_ref = dict_links_pl[p1]

# driver.get('https://energybase.ru/pipeline/espo-1-taishet-skovorodino')
driver.get(pl_ref)
ps = driver.page_source

In [105]:
soup = BeautifulSoup(ps,'html.parser')

In [115]:
st_info_0, st_info_1, st_info_2, st_info_3 = {}, {}, {}, {}

tab_1 = soup.find('div',class_="object-presentation-card__items").contents
for el in tab_1:
    try:
        st_info_1[el.contents[0].text] = el.contents[1].text
    except:
        st_info_1[el.contents[0].text] = el.contents[1]

tab_2 = soup.find('div',class_="col object-presentation-card__items").contents
for el in tab_2:
    try:
        st_info_2[el.contents[0].text] = el.contents[1].text
    except:
        st_info_2[el.contents[0].text] = el.contents[1]

tab_3 = soup.find_all('div', class_="secondary-properties-widget__row")
for el in tab_3:
    try:
        st_info_3[el.contents[0]] = el.contents[2].text
    except:
        st_info_3[el.contents[0]] = el.contents[2]

st_info_0[soup.find('title').text] = [st_info_1, st_info_2, st_info_3]

print(st_info_0)

{'Нефтепровод КТК Тенгиз - Новороссийск': [{'Протяженность': '1560 км', 'Мощность': '43 млн. тонн/год', 'Тип в зависимости от транспортируемой среды': 'Нефтепровод', 'Диаметр труб': '1020 мм'}, {'ID объекта': 'eb#pipe00035', 'Компания': 'АО "Каспийский трубопроводный консорциум-Р" (КТК-Р)'}, {'Давление': '6 МПа', 'Год начала строительства': '1992 ', 'Год ввода в эксплуатацию': '2001 ', 'Состояние': 'в эксплуатации ', 'Тип в зависимости от вида прокладки': 'Подземный '}]}


In [139]:
headers_dict = {'headers':[st_info_0, st_info_1, st_info_2, st_info_3]}

In [129]:
list_st = []
for card in soup.find_all('div',class_='object-card'):
    list_st.append(card.find('div',class_='object-card__title').a.get('href'))

In [130]:
len(list_st)

15

In [131]:
# list_st

In [132]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
# s = Service('/Users/vladimirkosarev/my_selenium/chromedriver')
# driver = webdriver.Chrome(service=s, options=chrome_options)

In [52]:
list_md = []
for link in list_st[13:]:
    driver = webdriver.Chrome(chrome_options=chrome_options)
    driver.get(link)
    ps = driver.page_source

    soup = BeautifulSoup(ps,'html.parser')

    md_0, md_1, md_2, md_3 = {}, {}, {}, {}

    tab_1 = soup.find('div',class_="object-presentation-card__items").contents
    for el in tab_1:
        try:
            md_1[el.contents[0].text] = el.contents[1].text
        except:
            md_1[el.contents[0].text] = el.contents[1]

    tab_2 = soup.find('div',class_="col object-presentation-card__items").contents
    for el in tab_2:
        try:
            md_2[el.contents[0].text] = el.contents[1].text
        except:
            md_2[el.contents[0].text] = el.contents[1]

    tab_3 = soup.find_all('div', class_="secondary-properties-widget__row")
    for el in tab_3:
        try:
            md_3[el.contents[0]] = el.contents[2].text
        except:
            md_3[el.contents[0]] = el.contents[2]

    md_0[soup.find('title').text] = [md_1, md_2, md_3]

    print(md_0)
    list_md.append(md_0)

    driver.quit()

In [140]:
pipeline_dict = {'items':list_md}
dict_to_json = {p1:[headers_dict, pipeline_dict]}

In [256]:
with open('data/transneft_vostok.json', 'w') as fp:
    json.dump(pipeline_dict, fp)

# Opening JSON file

In [33]:
f = open('data/transneft_vostok.json')
m_dict = json.load(f)

In [1]:
# m_dict to df

key = []
val = []
for di in m_dict['Транснефть – Восток']:

    key.append(list(di.keys())[0])
    val.append(list(di.values())[0][1])

df = pd.DataFrame.from_dict(val)
df['name'] = key



df['coord'] = df['Координаты'].apply(lambda x: re.findall("\d+\.\d+",x))

df